#### Test att hämta bilder och söka i SPA
* [Denna notebook](https://github.com/salgo60/spa2Commons/blob/main/Notebook/SPA%20Litteraturbanken.ipynb) 

Tanken att se vilka möjligheter som finns
* skapa ett script --> ett klick för att ladda upp bilder se test [spa2commons](https://commons.wikimedia.org/wiki/User:Salgo60/spa2commons.js) - GITHUB [salgo60/spa2Commons](https://github.com/salgo60/spa2Commons) 
* att ta en plats som en kyrkogård eller en koppling Litteraturbanken och se vad som kan matchas
* att stämma av länkar SPA -> WIkipedia att dessa finns i WIkidata med länk tillbaka via [P4819](https://www.wikidata.org/wiki/Property:P4819?uselang=sv)

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2021-09-24 15:34:56.182442


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 

url= "https://portrattarkiv.se/endpoints/latest.php"
url= "https://xn--portrttarkiv-kcb.se/endpoints/search.php"
url= "https://portrattarkiv.se/endpoints/search.php"  


## Icke kopplade personer P4819 hos Litteraturbanken P5101

Testar att söka med mer parametrar flr att få bättre precision i rankingen
* personer hos Litteraturbanken [Property:P5101](https://www.wikidata.org/wiki/Property:P5101?uselang=sv)


In [8]:
# query to get people with no picture in Litteraturbanken
endpoint_url = "https://query.wikidata.org/sparql"
import sys
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

query = """SELECT (CONCAT (?itemLabel," ",str(year(?birthDayWD))) AS ?search)
?birthDayWD
?firstnameLabel ?lastnameLabel
?item ?itemLabel 
(str(year(?birthDayWD)) AS ?BirthYearWD)
  WHERE {
  ?item wdt:P5101 ?Litt. 
  minus {?item wdt:P4819 ?c }
  minus {?item wdt:P18 ?c }
  OPTIONAL { ?item wdt:P735 ?firstname. }
  OPTIONAL { ?item wdt:P734 ?lastname. }
  OPTIONAL { ?item wdt:P569 ?birthDayWD. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv", "en". }
} """
def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)


SPALittdf = get_sparql_dataframe(endpoint_url, query)
SPALittdf.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1045 entries, 0 to 1044
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   search          1000 non-null   object
 1   birthDayWD      1001 non-null   object
 2   firstnameLabel  895 non-null    object
 3   lastnameLabel   535 non-null    object
 4   item            1045 non-null   object
 5   itemLabel       1045 non-null   object
 6   BirthYearWD     1000 non-null   object
dtypes: object(7)
memory usage: 57.3+ KB


In [9]:
SPALittdf

,search,birthDayWD,firstnameLabel,lastnameLabel,item,itemLabel,BirthYearWD
0,Heinrich von Langenstein 1325,1325-01-01T00:00:00Z,Henry,None,http://www.wikidata.org/entity/Q71997,Heinrich von Langenstein,1325
1,Adam av Bremen 1100,1100-01-01T00:00:00Z,Adam,None,http://www.wikidata.org/entity/Q109244,Adam av Bremen,1100
2,Antipatros från Sidon -200,-0200-01-01T00:00:00Z,Antypas,None,http://www.wikidata.org/entity/Q114280,Antipatros från Sidon,-200
3,None,t2434827672,Anita,None,http://www.wikidata.org/entity/Q241120,Anyte,None
4,Samuel Columbus 1642,1642-04-24T00:00:00Z,Samuel,Columbus,http://www.wikidata.org/entity/Q246312,Samuel Columbus,1642
...,...,...,...,...,...,...,...
1040,Jonas Erik Roselius 1813,1813-03-29T00:00:00Z,Erik,Roselius,http://www.wikidata.org/entity/Q108623545,Jonas Erik Roselius,1813
1041,Jonas Erik Roselius 1813,1813-03-29T00:00:00Z,Jonas,Roselius,http://www.wikidata.org/entity/Q108623545,Jonas Erik Roselius,1813
1042,Hjalmar Gustaf Ferdinand Höglund 1865,1865-09-06T00:00:00Z,None,None,http://www.wikidata.org/entity/Q108641487,Hjalmar Gustaf Ferdinand Höglund,1865
1043,Gustaf Carlsson Leijonhufvud 1861,1861-07-11T00:00:00Z,None,None,http://www.wikidata.org/entity/Q108649719,Gustaf Carlsson Leijonhufvud,1861


In [10]:
from wikidata.client import Client
from tqdm.notebook import tqdm 
import urllib3, json
http = urllib3.PoolManager() 
SPAdetail = "https://portrattarkiv.se/details/"
url= "https://portrattarkiv.se/endpoints/search.php"  
urlbasePic = "https://portrattarkiv.se/endpoints/file.php?id="

def getdfScore(panddf):
    dfScore = pd.DataFrame(columns=['all','wd','spaid','score',
                                    'FirstNameWD','LastNameWD','BirthDayWD',
                                    'FirstNameSPA','LastNameSPA','BirthDaySPA'])
    #for index, row  in SPALittdf.iterrows():
    for index, row in tqdm(panddf.iterrows(),total=panddf.shape[0]):
    
        allString=row["search"]
        wd=row["item"]
        print("\n",allString,wd)
        encoded_body = json.dumps({
            "limit": "5",
            "from": "0",
            "firstname" : row["firstnameLabel"], 
            "lastname" : row["lastnameLabel"], 
            "birthyear":row["BirthYearWD"],            
            "all":allString
            
        })
    
        r = http.request('POST', url,
                 headers={'Content-Type': 'application/json'},
                 body=encoded_body)

        if r.status != 200:
            print(r.status)
            continue
    
        data = json.loads(r.data.decode('utf-8'),) 
        urls = []

        for h in data["hits"]["hits"]:
            spaid = h["_id"]
            #print(h)
            source = h["_source"]
            try:
                urlPicture = urlbasePic + spaid
                urls.append(urlPicture)
                score = h["_score"]
                FirstNameSPA = source["FirstName"]
                LastNameSPA = source["LastName"]
                BirthYearSPA = source["BirthYear"]
                print("\t\t",score,FirstNameSPA, " ", LastNameSPA, " - ", BirthYearSPA,SPAdetail+spaid, "\t", )
                dfScore = dfScore.append({
                'all' : allString,
                'wd' : wd,
                'spaid' : spaid,
                'score' : score,
                'FirstNameWD' : row["firstnameLabel"],
                'LastNameWD' : row["lastnameLabel"],
                'BirthYearWD' : row["BirthYearWD"],
                'FirstNameSPA' : FirstNameSPA,
                'LastNameSPA' : LastNameSPA,
                'BirthYearSPA' : BirthYearSPA
                },ignore_index=True)
            except Exception as e:
                print(f"{type(e).__name__} at line {e.__traceback__.tb_lineno} of {__file__}: {e}")
    return dfScore

In [11]:
dfScoreSPALitt = getdfScore(SPALittdf)
dfScoreSPALitt.info()

  0%|          | 0/1045 [00:00<?, ?it/s]


 Heinrich von Langenstein 1325 http://www.wikidata.org/entity/Q71997

 Adam av Bremen 1100 http://www.wikidata.org/entity/Q109244

 Antipatros från Sidon -200 http://www.wikidata.org/entity/Q114280

 None http://www.wikidata.org/entity/Q241120
		 16.367397 Anita   Björk  -  1923 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAB2aeA 	
		 15.220704 Anita   Nathorst  -  1894 https://portrattarkiv.se/details/wbbNj8LLWMAAAAAAAAApcQ 	
		 15.133949 Anita   Lund  -  1944 https://portrattarkiv.se/details/rkIpfGqHXrAAAAAAAACG3A 	
		 14.16946 Anita   Björk  -  1923 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABkKLQ 	
		 13.731096 Anita   Elmhagen  -  1937 https://portrattarkiv.se/details/TR3yO_GV8eAAAAAAAAB1eQ 	

 Samuel Columbus 1642 http://www.wikidata.org/entity/Q246312

 Lars Wivallius 1605 http://www.wikidata.org/entity/Q246991
		 21.096134 Hans   von Königsmark  -  1605 https://portrattarkiv.se/details/4e1scVvZ-EAAAAAAAAAePg 	

 Bakchylides -515 http://www.wikidata.org/entity/Q31068


 Emmy Köhler 1858 http://www.wikidata.org/entity/Q3427152

 Emmy Köhler 1858 http://www.wikidata.org/entity/Q3427152

 Gunnar Ohrlander 1939 http://www.wikidata.org/entity/Q3429810

 Martin Kylhammar 1954 http://www.wikidata.org/entity/Q3430397
		 10.441549 Ewa   Berggren  -  1954 https://portrattarkiv.se/details/TNQaWo324IAAAAAAAAC6zQ 	
		 10.092919 Els-Marie   Sundberg  -  1954 https://portrattarkiv.se/details/YB0QHyfj0hAAAAAAAACQZQ 	
		 10.030746 Britt-Carin   Hjort  -  1954 https://portrattarkiv.se/details/j6S8rJvx9lAAAAAAAABavA 	

 Karl Eneberg 1841 http://www.wikidata.org/entity/Q3431539
		 17.473482 Carl   Nyberg  -  1841 https://portrattarkiv.se/details/wbbNj8LLWMAAAAAAAABOiQ 	

 Johan Fredrik Peringskiöld 1689 http://www.wikidata.org/entity/Q3481570

 Nils Ragvaldsson 1380 http://www.wikidata.org/entity/Q3877027

 Olaus Martini 1557 http://www.wikidata.org/entity/Q3881468

 Ulf Peter Hallberg 1953 http://www.wikidata.org/entity/Q4003029

 Ulf Peter Hallberg 1953 http://www.wi

		 15.00275 Alma   Lundberg  -  1850 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACaIOg 	

 Johan Eggert Åberg 1820 http://www.wikidata.org/entity/Q5410208

 Hilma Montell 1853 http://www.wikidata.org/entity/Q5478536

 Hilma Montell 1853 http://www.wikidata.org/entity/Q5478536

 Theodor Lindh 1833 http://www.wikidata.org/entity/Q5494968

 Theodor Lindh 1833 http://www.wikidata.org/entity/Q5494968

 George Carter 1737 http://www.wikidata.org/entity/Q5537718

 Johann Ahlich 1650 http://www.wikidata.org/entity/Q5544058

 Bo Alström 1937 http://www.wikidata.org/entity/Q5547974

 Bo Alström 1937 http://www.wikidata.org/entity/Q5547974

 Bo Alström 1937 http://www.wikidata.org/entity/Q5547974

 Martin Altén 1764 http://www.wikidata.org/entity/Q5548052

 Djos Per Andersson 1814 http://www.wikidata.org/entity/Q5554362
		 24.983807 Johan   Andersson  -  1814 https://portrattarkiv.se/details/28K7pO-SRrAAAAAAAACjUw 	
		 24.957062 Johanna   Andersson  -  1814 https://portrattarkiv.se/details/


 Bernhard Cronholm 1813 http://www.wikidata.org/entity/Q5613494

 Nils Hufwedsson Dal 1690 http://www.wikidata.org/entity/Q5615947

 Sveno Dalius 1604 http://www.wikidata.org/entity/Q5616104

 Isak af Darelli 1756 http://www.wikidata.org/entity/Q5617156

 Gunnar D Hansson 1945 http://www.wikidata.org/entity/Q5618976

 Carl Erik Deléen 1767 http://www.wikidata.org/entity/Q5619757
		 45.349857 Carl   Deleen  -  1767 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAB2--A 	
		 31.739677 Carl   Deleen  -  1767 https://portrattarkiv.se/details/4e1scVvZ-EAAAAAAAAAQKQ 	
		 27.198086 Carl   Deleen  -  1767 https://portrattarkiv.se/details/_P7S9YzQCjAAAAAAAAAvcQ 	
		 26.549116 Carl   Stiernswärd  -  1767 https://portrattarkiv.se/details/mLh5P0pzYFAAAAAAAAA2PQ 	
		 24.33424 Carl   Wallman  -  1767 https://portrattarkiv.se/details/ciLMGSqrYjAAAAAAAAAYdg 	

 Adolf Fredrik Ristell 1744 http://www.wikidata.org/entity/Q6073623

 Gustaf Rosén 1772 http://www.wikidata.org/entity/Q6079264

 Claes Rosen

...

		 6.2641754 Ingegerd   Elm  -  1936 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABzqkw 	

 Johannes Granlund 1841 http://www.wikidata.org/entity/Q18239228

 Anders Gottlieb Herkepæus 1728 http://www.wikidata.org/entity/Q18242515

 Sven Lundblad 1789 http://www.wikidata.org/entity/Q18244065

 Sophie Nordling 1839 http://www.wikidata.org/entity/Q18244725

 Nils Skytte 1656 http://www.wikidata.org/entity/Q18245896

 Evald Ziervogel 1728 http://www.wikidata.org/entity/Q18275031

 Björn Meidal 1948 http://www.wikidata.org/entity/Q18638928

 Mnasalcas -299 http://www.wikidata.org/entity/Q19053390

 Paul Berf 1963 http://www.wikidata.org/entity/Q19298273

 Jacobus Petri Rondeletius 1580 http://www.wikidata.org/entity/Q19604500

 Drude von der Fehr 1945 http://www.wikidata.org/entity/Q19707793
		 7.648846 Lars   Ström  -  1945 https://portrattarkiv.se/details/YB0QHyfj0hAAAAAAAABUYA 	
		 7.488483 Marie   Ahrle  -  1945 https://portrattarkiv.se/details/28K7pO-SRrAAAAAAAACqYQ 	
		 7.4790716

		 36.017326 Karin   Nyström  -  1854 https://portrattarkiv.se/details/wbbNj8LLWMAAAAAAAAB-wg 	
		 31.309345 Johanna   Ström  -  1854 https://portrattarkiv.se/details/4e1scVvZ-EAAAAAAAAASCA 	

 Max Krook 1860 http://www.wikidata.org/entity/Q11881850

 Nenne Moberg 1868 http://www.wikidata.org/entity/Q11884183
		 34.503357 Carl   Moberg  -  1868 https://portrattarkiv.se/details/I_2wtd5VefAAAAAAAAAloA 	
		 33.33469 Olof   Moberg  -  1868 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABFpHA 	
		 27.217817 Olof   Moberg  -  1868 https://portrattarkiv.se/details/seuNpYu6DKAAAAAAAAAOnA 	
		 14.282627 Karl   Ekberg  -  1868 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAB6oqA 	
		 14.052754 Johan   Ekberg  -  1868 https://portrattarkiv.se/details/TR3yO_GV8eAAAAAAAABQTw 	

 Olga Torckell 1859 http://www.wikidata.org/entity/Q11885364

 Oscar Behm 1867 http://www.wikidata.org/entity/Q11885835
		 22.154766 Oscar   Böhm  -  1867 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA2LCA 	

 R. 


 Sven-Bertil Jansson 1935 http://www.wikidata.org/entity/Q74250430
		 24.278938 Bertil   Schill  -  1935 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABhAAg 	
		 23.983152 Bertil   Hansson  -  1935 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABNKvg 	
		 23.900475 Bertil   Gärdh  -  1935 https://portrattarkiv.se/details/1ha4xkDj-uAAAAAAAABKPA 	
		 23.453901 Bertil   Thorvaldsson  -  1935 https://portrattarkiv.se/details/NhqpvvI9tpAAAAAAAAAz9A 	
		 23.361477 Bertil   Thorvaldsson  -  1935 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABhZTA 	

 Birgitta Bergsten 1943 http://www.wikidata.org/entity/Q75714539
		 29.39883 Birgitta   Bergdahl  -  1943 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABhhYw 	
		 29.241123 Birgitta   Ericson  -  1943 https://portrattarkiv.se/details/TR3yO_GV8eAAAAAAAAANZQ 	
		 29.128866 Birgitta   Bergdahl  -  1943 https://portrattarkiv.se/details/TNQaWo324IAAAAAAAAD3Vw 	
		 27.905945 Birgitta   Carlsson-Nilsson  -  1943 https://portrattarkiv.se/d


 None http://www.wikidata.org/entity/Q108393463
400

 Ulrik Uddman 1828 http://www.wikidata.org/entity/Q108393816
		 15.486144 Ulrika   Lindgren  -  1828 https://portrattarkiv.se/details/4e1scVvZ-EAAAAAAAAAXig 	
		 14.681553 Ulrika   Lindgren  -  1828 https://portrattarkiv.se/details/rkIpfGqHXrAAAAAAAADdxA 	
		 14.621899 Ulrika   Lindgren  -  1828 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACYnnQ 	
		 14.113117 Erik   Åsell  -  1828 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABKjBA 	
		 14.113117 Erik   Åsell  -  1828 https://portrattarkiv.se/details/xnPIH5lXePAAAAAAAAAWlw 	

 Ulrik Uddman 1828 http://www.wikidata.org/entity/Q108393816
		 16.572443 Mattias   Westerlund  -  1828 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABHAQQ 	
		 15.868018 Maria   Nordlander  -  1828 https://portrattarkiv.se/details/wbbNj8LLWMAAAAAAAAEFjw 	
		 15.357488 Matts   Westerlund  -  1828 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACdz1g 	
		 14.93375 Maria   Levgren  -  1828 https:/

		 22.399406 Märta   Leijonhufvud  -  1861 https://portrattarkiv.se/details/mLh5P0pzYFAAAAAAAABBhw 	
		 18.119356 Carl   Carlsson  -  1861 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA6GuQ 	
		 17.742584 Gustaf   Carlsson  -  1861 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABGCdw 	
		 17.422625 Hanna   Feuk  -  1861 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACZjcA 	

 Adolf August Schéele 1827 http://www.wikidata.org/entity/Q108652294
		 21.50925 Karl   Lundberg  -  1827 https://portrattarkiv.se/details/4e1scVvZ-EAAAAAAAAAeEQ 	
		 18.882936 August   Hillberg  -  1827 https://portrattarkiv.se/details/mLh5P0pzYFAAAAAAAAAnHw 	
		 18.57795 Carl   Lagerfelt  -  1827 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABMGcw 	
		 18.545435 Carl   Lagerfelt  -  1827 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABMGbw 	
		 18.486698 August   Hillberg  -  1827 https://portrattarkiv.se/details/4e1scVvZ-EAAAAAAAAAC5g 	
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1511 ent

In [24]:
dfScoreSPALitt.sort_values('score', ascending=False)
dfScoreSPALitt.groupby("wd")

In [25]:
#Group by WIkidata Q number and get highest score 

#create a helper column with max
dfScoreSPALitt['max'] = dfScoreSPALitt.groupby('wd')['score'].transform('max')  

dfScoreSPALittMax =  dfScoreSPALitt.groupby('wd')['score','max','spaid','FirstName','LastName','BirthYear'].apply(lambda x: x.nlargest(1, columns=['score'])).sort_values(['max','wd','score'], ascending=False)
dfScoreSPALittMax   


<ipython-input-25-53d508d08f0b>:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dfScoreSPALittMax =  dfScoreSPALitt.groupby('wd')['score','max','spaid','FirstName','LastName','BirthYear'].apply(lambda x: x.nlargest(1, columns=['score'])).sort_values(['max','wd','score'], ascending=False)


,,score,max,spaid,FirstName,LastName,BirthYear
wd,,,,,,,
http://www.wikidata.org/entity/Q9191,3283,56.872803,56.872803,sj9PGLAlnmUAAAAAABzpOQ,René,Descartes,1596
http://www.wikidata.org/entity/Q22670927,3902,54.135574,54.135574,sj9PGLAlnmUAAAAAACYy8A,Anna,Gustafsdotter-Banér,1585
http://www.wikidata.org/entity/Q1527703,292,51.183067,51.183067,WQknIBFpuuAAAAAAAANgZA,Jakob,Björnståhl,1731
http://www.wikidata.org/entity/Q6000000,5589,48.400414,48.400414,TR3yO_GV8eAAAAAAAAAKVg,Erik,Emporagrius,1606
http://www.wikidata.org/entity/Q4945390,879,48.209740,48.209740,sj9PGLAlnmUAAAAAACYzcg,Dorothea,Dunckel,1799
http://www.wikidata.org/entity/Q1373367,237,48.102306,48.102306,sj9PGLAlnmUAAAAAAB5W0Q,Olof,von-Dalin,1708
http://www.wikidata.org/entity/Q5785026,4647,47.801285,47.801285,sj9PGLAlnmUAAAAAAB4cOA,Carl,Hallman,1732
http://www.wikidata.org/entity/Q2693785,1794,47.698288,47.698288,VaM-WW8ntnAAAAAAAAABUQ,Johan III,Sobieski,1629
http://www.wikidata.org/entity/Q5613778,2538,47.457794,47.457794,4e1scVvZ-EAAAAAAAAAHtg,Johan,Crusenstolpe,1801


In [26]:
dfScoreSPALittMax.to_csv("Littraturbanken_max.csv") 